# This notebook shows how to estimate a spot with a gaussian model.
* It is possible to fit a spot with one or several gaussians.
* Differents algorithmes are available.

In [1]:
%matplotlib notebook

import multiprocessing.pool
import pathlib

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc import Diagram
from laueimproc.io.download import get_samples  # gives access to the dataset

### Firstly, we have to create the diagrams from images
* To see all the ways to create a diagram, please have a look to the `init_diagram` notebook.
* Take care having enough RAM and swappiness to load all the images, otherwise it would bias the peak search time.

In [2]:
all_files = sorted(get_samples().iterdir())  # the list of all images path
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots()

## Fit by one gaussian

In [3]:
# very fast fit with "expectation maximization" algorithm
mean_em, cov_em, infodict_em = diagrams[0].fit_gaussian_em()  # like pca and barycenter
print("shape mean:", mean_em.shape)  # the position i, j of the center of the gaussian for each spot
print("shape cov:", cov_em.shape)  # the covariance matrix of the gaussian for each spot
assert infodict_em == {}  # empty because no parameters provided
*_, infodict_em = diagrams[0].fit_gaussian_em(aic=True, bic=True, eigtheta=True, log_likelihood=True)
print(sorted(infodict_em))  # more informations
*_, infodict_em = diagrams[0].fit_gaussian_em(photon_density=10900.0, tol=True)
print(sorted(infodict_em))  # mean standard deviation estimator

# fast fit with classical loss minimization
mean_mse, cov_mse, mag_mse, infodict_mse = diagrams[0].fit_gaussian(loss="mse")
print("shape mean:", mean_em.shape)  # the position i, j of the center of the gaussian for each spot
print("shape cov:", cov_em.shape)  # the covariance matrix of the gaussian for each spot

shape mean: torch.Size([135, 2])
shape cov: torch.Size([135, 2, 2])
['aic', 'bic', 'eigtheta', 'log_likelihood']
['tol']
tensor([[[ 1.0236e-05, -1.3019e-03, -8.1089e-05,  ..., -3.0306e-05,
          -1.5240e-04, -1.2190e-04],
         [ 1.0470e-04, -1.2361e-03, -3.1864e-04,  ..., -2.0962e-04,
          -1.4897e-04, -5.7774e-05],
         [ 6.0726e-04, -1.6344e-02, -1.5077e-02,  ..., -2.0199e-04,
          -1.4473e-04,  3.4889e-05],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 2.0895e-02,  8.0950e-02,  5.2528e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 4.5238e-02,  1.9398e-01,  1.3828e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 5.7524e-02, 

NotImplementedError: 

ERROR:root:
ERROR:root:


tensor([[[ 4.7143e-03,  1.9653e-02,  3.9676e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.3872e-02,  5.4193e-02,  1.0883e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.1484e-02,  8.0566e-02,  1.5914e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 6.0114e-25,  2.0480e-24,  3.8551e-24,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.3570e-28,  4.6037e-28,  8.6298e-28,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.6353e-32,  5.5247e-32,  1.0313e-31,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-4.9823e-04,  8.7509e-04,  4.0113e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 6.9109e-04,  6.2419e-03,  1.8553e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 3.5882e-03,  1.9145e-02,  5.1011e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 2.1248e-14,  4

tensor([[[-5.1052e-04,  1.8320e-04,  1.2536e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.1831e-04,  2.7284e-03,  6.1771e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.4497e-03,  7.7576e-03,  1.5446e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 1.8391e-14,  3.2664e-14,  4.5662e-14,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.7515e-16,  4.7985e-16,  6.5867e-16,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 3.0478e-18,  5.2192e-18,  7.0345e-18,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 4.1562e-03,  1.3916e-02,  2.1203e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.1477e-02,  3.1944e-02,  4.9220e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.9260e-02,  5.0512e-02,  7.8682e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 4.8446e-17,  1

...

tensor([[[ 1.8816e-02,  4.7364e-02,  5.5309e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 8.6169e-02,  2.4416e-01,  3.2322e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 5.6878e-02,  1.8273e-01,  2.7345e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 2.0081e-02,  2.5702e-02,  1.3524e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 8.4260e-02,  1.5463e-01,  1.1635e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0081e-01,  2.6561e-01,  2.8684e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0

ERROR:root:


tensor([[[-1.5259e-04, -9.1553e-05, -1.8311e-04,  ...,  3.1490e-19,
           1.3640e-19,  5.7709e-20],
         [ 4.1894e-05,  6.2937e-05, -4.2607e-06,  ...,  2.4634e-10,
           1.0784e-10,  4.6114e-11],
         [ 2.9016e-02,  3.8382e-02,  4.9810e-02,  ...,  1.9288e-07,
           8.5338e-08,  3.6881e-08],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 2.3069e-04, -8.5360e-04,  2.3235e-04,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.6217e-03,  7.9900e-04,  2.3102e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 6.8155e-03, -7.7877e-03, -5.8789e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0

ERROR:root:


tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],